# extrakcia dňa z html súborov a uloženie do csv  

skript extrahuje názov dňa (h2 tag) z oficiálnych predgenerovaných html súborov (dostupných na https://lc.kbs.sk/).  

Na začiatku je potrebné stiahnuť zip archív predgenerovaných súborov a rozbaliť do adresára `'./data_html/'`

In [6]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [7]:
data_dir = './data_html/'

In [8]:
# List .html files from data directory with file name length 15 (meaning particular date like: '2023-01-01.html')
# poznámka: 
# keďže cirkevný rok začína adventom, sú v zip archíve aj dni z predošlého roka (2022)
# preto som pre názornosť dal podmienku, aby bral len aktuálny rok 2023 (file.startswith('2023'))
html_files = [ file for file in os.listdir(data_dir) if file.startswith('2023') and file.endswith(".html") and len(file)==15 ]

len(html_files) 

365

In [9]:
dictionary = { }

for fname in html_files:
    with open(data_dir+fname, encoding='utf-8') as f:
        s = f.read()    
        
    soup = BeautifulSoup(s,'lxml') #     
    ciel = soup.h2.text
    vysledny_string = ciel.replace('\n',' ').replace('  ',' ').strip()
    datum = fname.replace('.html','')
    dictionary[datum] = vysledny_string


c:\Python\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [ ]:
dictionary

In [11]:
df = pd.DataFrame.from_dict(dictionary, orient='index', columns=['nazov'])
df

,nazov
2023-01-01,PRIKÁZANÁ SLÁVNOSŤ: Panny Márie Bohorodičky [B...
2023-01-02,"Svätých Bazila Veľkého a Gregora Nazianzského,..."
2023-01-03,Utorok vo Vianočnom období [Biela] alebo Najsv...
2023-01-04,Streda vo Vianočnom období [Biela]
2023-01-05,Štvrtok vo Vianočnom období [Biela]
...,...
2023-12-27,"Svätého Jána, apoštola a evanjelistu [Biela] (..."
2023-12-28,"Svätých Neviniatok, mučeníkov [Červená] (sviatok)"
2023-12-29,Piaty deň v oktáve narodenia Pána [Biela]
2023-12-30,Šiesty deň v oktáve narodenia Pána [Biela]


In [12]:
df.to_csv('output.csv')